# Creating a Reconstruction Error Explainer:

### Imports:

In [107]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf

### Reconstruction Error Explainer Class:

In [96]:
class ReconstructionErrorExplainer:
    """
    Decomposed an Autoencoders reconstruction error into:
        - Feature-level errors
        - Percentage contribution per feature
        - Optional behavioral group-level aggregation
    """
    
    def __init__(self, feature_names: list, feature_groups: dict | None=None) -> None:
        """
        Initializes the explainer.
        
        Args:
            feature_names: An ordered list of feature names matching model input order
            feature_groups: A dictionary mapping of group names to feature names
            
        Returns:
            None:
        """
        self.feature_names = feature_names
        self.feature_groups = feature_groups
        self.feature_index_map = {name:idx for idx, name in enumerate(feature_names)}
        
        
    def compute_feature_error(self, input_data: np.ndarray, reconstructed_data: np.ndarray) -> np.ndarray:
        """
        Computes squared reconstruction error per feature.
        
        Args:
            input_data: The original input features of shape: (n_samples, n_features)
            reconstructed_data: The reconstructed features generated by autoencoders of shape: (n_samples, n_features)
            
        Returns:
            np.ndarray: The squared error per feature of shape: (n_samples, n_features)
        """
        return np.square(input_data - reconstructed_data)
    
    
    def compute_total_error(self, feature_error: np.ndarray) -> np.ndarray:
        """
        Computes the total reconstruction error per sample.
        
        Args:
            feature_error: An array containing feature-level errors
            
        Returns:
            np.ndarray: The total reconstruction error per sample
        """
        return np.sum(feature_error, axis=1)
    
    
    def compute_contribution_ratio(self, feature_error: np.ndarray, total_error: np.ndarray) -> np.ndarray:
        """
        Computes the percentage contributions fo each feature to the total error.
        
        Args:
            feature_error: An array containing feature-level errors
            total_errors: An array containing total reconstruction error per sample
            
        Returns:
            np.ndarray: The contribition ratio of the shape: (n_samples, n_features)
        """
        # Ensuring there's no division by 0
        total_error_safe = np.where(total_error == 0, 1e-8, total_error)
        
        contribution_ratio = feature_error / total_error_safe[:, np.newaxis]
        return contribution_ratio
    
    
    def compute_group_error(self, feature_error: np.ndarray) -> np.ndarray | None:
        """
        Aggregates feature errors by behavioral groups.
        
        Args:
            feature_error: An array containing feature-level errors
            
        Returns:
            np.ndarray: An aggregate array of shape: (n_samples, n_groups)
        """
        if self.feature_groups is None:
            return None
        
        group_errors = []
        
        for group, features in self.feature_groups.items():
            indices = [self.feature_index_map[f] for f in features if f in self.feature_index_map]
            
            group_error = np.sum(feature_error[:, indices], axis=1)
            group_errors.append(group_error)
            
        return np.vstack(group_errors).T
    
    
    def explain(self, input_data: np.ndarray, model: tf.keras.Model) -> dict:
        """
        Generates a full reconstruction explaination.
        
        Args:
            input_data: The input data
            model: A trained autoencoder model
            
        Returns:
            dict: A dictionary containing feature error, total error, contribution ration, and group error
        """
        x_pred = model.predict(input_data, verbose=0)
        
        feature_error = self.compute_feature_error(input_data, x_pred)
        total_error = self.compute_total_error(feature_error)
        contribution_ratio = self.compute_contribution_ratio(feature_error, total_error)
        group_error = self.compute_group_error(feature_error)
        
        error_dict = {
            "feature_error": feature_error,
            "total_error": total_error,
            "contribution_ratio": contribution_ratio,
            "group_error": group_error
        }
        
        return error_dict
    
    
    def explain_to_df(self, input_data: np.ndarray, model: tf.keras.Model, metadata: pd.DataFrame | None=None, include_feat_err: bool=True, include_contributions: bool=True) -> pd.DataFrame:
        """
        Generates a structured Pandas DataFrame containing reconstruction explainations.
        
        Args:
            input_data: The scaled input data
            model: A trained autoencoder model
            metadata: Optional metadata columns
            include_feat_err: Includes raw per-feature reconstruction errors
            include_contributions: Inlcudes per-feature contribution ratios
            
        Returns:
            pd.DataFrame: A structured DataFrame containing metadata, total error, group-level errors, and optional feature-level details
        """
        # Computing reconstruction explainations
        results = self.explain(input_data, model)
        
        feature_error = results["feature_error"]
        total_error = results["total_error"]
        contribution_ratio = results["contribution_ratio"]
        group_error = results["group_error"]
        
        error_df = pd.DataFrame()
        
        # Adding metadata
        if metadata is not None:
            error_df = metadata.reset_index(drop=True).copy()
            
        # Add total reconstruction error
        error_df["total_reconstruction_error"] = total_error
            
        # Adding group-level errors if defined
        if (self.feature_groups is not None) and (group_error is not None):
            for idx, group in enumerate(self.feature_groups.keys()):
                error_df[f"group_error_{group}"] = group_error[:, idx]
                
        # Adding feature-level errors if true
        if include_feat_err:
            for idx, feature in enumerate(self.feature_names):
                error_df[f"error_{feature}"] = feature_error[:, idx]
                
        # Adding contribution ratios if true
        if include_contributions:
            for idx, feature in enumerate(self.feature_names):
                error_df[f"contribution_{feature}"] = contribution_ratio[:, idx]
                
        return error_df

### Importing UEBA-Enhanced Dataset and Trained Autoencoder:

In [33]:
ueba_dataset = pd.read_csv(r"processed_datasets\ueba_dataset.csv", index_col=0)

In [ ]:
feature_names = list(ueba_dataset.drop(columns=["user", "pc", "day"]).columns)
ueba_matrix = ueba_dataset.drop(columns=["user", "pc", "day"]).values

In [39]:
ae = tf.keras.models.load_model(r"encoders\encoder_model_1\autoencoder_model.keras")

### Generating Full Reconstruction Explaination:

In [98]:
feature_groups = {
    "logon": ['logon_count', 'off_hours_logon', 'logon_count_zscore',
              'off_hours_logon_zscore', 'logon_count_rolling_delta',
              'off_hours_logon_rolling_delta', 'logoff_count',
              'logoff_count_zscore', 'logoff_count_rolling_delta'],
    
    "file": ['file_open_count', 'file_write_count', 'file_copy_count',
             'file_delete_count', 'unique_files_accessed', 'off_hours_files_accessed',
             'file_open_count_zscore', 'file_write_count_zscore', 'file_copy_count_zscore',
             'file_delete_count_zscore', 'unique_files_accessed_zscore',
             'off_hours_files_accessed_zscore','file_open_count_rolling_delta',
             'file_write_count_rolling_delta', 'file_copy_count_rolling_delta',
             'file_delete_count_rolling_delta', 'unique_files_accessed_rolling_delta',
             'off_hours_files_accessed_rolling_delta', 'usb_file_activity_flag'],
    
    "email": ['emails_sent', 'external_emails', 'off_hours_emails', 'emails_sent_zscore',
              'external_emails_zscore', 'off_hours_emails_zscore', 'emails_sent_rolling_delta',
              'external_emails_rolling_delta', 'off_hours_emails_rolling_delta', 'attachements_sent',
              'attachements_sent_rolling_delta','attachements_sent_zscore', 'external_comm_activity_flag',
              'unique_recipients', 'unique_recipients_rolling_delta', 'unique_recipients_zscore'],
    
    "device": ['usb_insert_count', 'usb_remove_count', 'off_hours_usb_usage', 'usb_insert_count_zscore',
               'usb_remove_count_zscore', 'off_hours_usb_usage_zscore', 'usb_insert_count_rolling_delta',
               'usb_remove_count_rolling_delta', 'off_hours_usb_usage_rolling_delta', 'usb_file_activity_flag']
}

In [99]:
explainer = ReconstructionErrorExplainer(feature_names=feature_names, feature_groups=feature_groups)

In [105]:
reconstruction_error = explainer.explain_to_df(ueba_matrix, ae, metadata=ueba_dataset[["user", "pc", "day"]])

C:\Users\loera\AppData\Local\Temp\ipykernel_24984\1534966398.py:166: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  error_df[f"contribution_{feature}"] = contribution_ratio[:, idx]
C:\Users\loera\AppData\Local\Temp\ipykernel_24984\1534966398.py:166: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  error_df[f"contribution_{feature}"] = contribution_ratio[:, idx]
C:\Users\loera\AppData\Local\Temp\ipykernel_24984\1534966398.py:166: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert`

In [106]:
reconstruction_error

,user,pc,day,total_reconstruction_error,group_error_logon,group_error_file,group_error_email,group_error_device,error_logon_count,error_logoff_count,...,contribution_usb_remove_count_rolling_delta,contribution_off_hours_usb_usage_rolling_delta,contribution_emails_sent_rolling_delta,contribution_unique_recipients_rolling_delta,contribution_external_emails_rolling_delta,contribution_attachements_sent_rolling_delta,contribution_off_hours_emails_rolling_delta,contribution_usb_file_activity_flag,contribution_off_hours_activity_flag,contribution_external_comm_activity_flag
0,aab0162,pc-6599,2010-01-04,68.203189,6.511592,9.251204,50.597944,2.884793,0.542574,1.403321,...,3.217004e-03,0.000378,0.003109,0.009307,0.011761,2.949682e-04,0.139050,0.017287,0.002004,0.001692
1,aab0162,pc-6599,2010-01-05,64.121186,9.053624,9.100881,44.784627,1.173859,1.110457,1.195250,...,2.346039e-03,0.000098,0.003382,0.008450,0.014016,1.696896e-03,0.014648,0.000354,0.000482,0.002379
2,aab0162,pc-6599,2010-01-06,82.450248,9.854220,17.189219,53.600398,1.560673,0.985287,2.357401,...,1.969509e-09,0.000092,0.001144,0.000859,0.000991,2.849795e-03,0.062452,0.002183,0.005164,0.009861
3,aab0162,pc-6599,2010-01-07,66.092342,7.893960,10.334823,46.858393,2.190616,0.808923,1.573427,...,2.870517e-04,0.000380,0.005083,0.013270,0.018409,8.088548e-04,0.115202,0.020497,0.002561,0.001280
4,aab0162,pc-6599,2010-01-08,68.243519,8.131239,8.983746,50.137954,1.204615,0.677157,1.243571,...,2.202536e-03,0.001499,0.000363,0.003335,0.031988,3.133765e-03,0.066222,0.004346,0.001210,0.002034
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1564764,zzo2997,pc-3120,2011-05-24,99.118546,2.121827,5.988074,89.469074,1.830506,0.021985,0.633764,...,5.114646e-04,0.000598,0.000227,0.033799,0.085851,1.460023e-07,0.010768,0.003708,0.000773,0.003910
1564765,zzo2997,pc-3120,2011-05-25,101.699538,3.096254,6.041491,90.648435,1.636764,0.074985,1.052651,...,1.348935e-03,0.000210,0.001968,0.001074,0.089945,1.829340e-04,0.007081,0.000017,0.002737,0.002287
1564766,zzo2997,pc-3120,2011-05-26,101.699932,2.746369,5.277735,92.241925,1.338759,0.038741,0.705734,...,2.405142e-04,0.000077,0.001966,0.001131,0.087211,1.772624e-04,0.011917,0.000059,0.000994,0.003584
1564767,zzo2997,pc-3120,2011-05-27,101.109335,3.066057,5.775404,90.384389,1.894207,0.283531,0.922565,...,2.730237e-03,0.000427,0.010673,0.000549,0.087944,2.377510e-04,0.005487,0.000468,0.000362,0.002699


### Saving the Reconstruction Error Table:

In [ ]:
# Defining save path
save_path = r"explainability\reconstruction_error"
os.makedirs(save_path, exist_ok=True)

In [109]:
reconstruction_error.to_csv(os.path.join(save_path, "reconstuction_error_table_1.csv"), index=False)